In [1]:
from dotenv import load_dotenv
import os

load_dotenv('../.env', override=True)
if os.getenv('OPENAI_API_KEY') is None:
    raise ValueError("OPENAI_API_KEY environment variable not set")
else:
    print("OPENAI_API_KEY is set")

OPENAI_API_KEY is set


In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

sys_msg = SystemMessage(
    content="You are a helpful assistant that speaks like Shakespeare."
)
human_msg = HumanMessage(
    content="What is the capital of France?"
)
messages = [sys_msg, human_msg]

In [3]:
response = llm.invoke(messages)
print(response)

content='The capital of France, good sir or madam, is Paris.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 28, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BWQMWOShChCkxQ7F31qr7j9wkfVY8', 'finish_reason': 'stop', 'logprobs': None} id='run-228b0403-4866-490d-bb76-011e4b96759c-0' usage_metadata={'input_tokens': 28, 'output_tokens': 15, 'total_tokens': 43, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [4]:
print(type(response))

<class 'langchain_core.messages.ai.AIMessage'>


## Test builtin tools

In [7]:
from core.agents.tools.builtins.wikipedia_tool import tool_function as wikipedia
print(type(wikipedia))
print(wikipedia)

<class 'langchain_core.tools.structured.StructuredTool'>
name='wikipedia' description='Query wiki. Use it to get factual information on: \n- historical figures, events, or places\n- scientific concepts\n- common knowledge\n- debunked myths' args_schema=<class 'langchain_core.utils.pydantic.wikipedia'> response_format='content_and_artifact' func=<function tool_function at 0x1162bc180>


In [8]:
llm_with_tools = llm.bind_tools([wikipedia])
sys_msg_tools = SystemMessage(
    content="You are a helpful research assistant that can use Wikipedia."
)
response_from_tools = llm_with_tools.invoke([sys_msg_tools, human_msg])

In [10]:
print(response_from_tools)

content='' additional_kwargs={'tool_calls': [{'id': 'call_qZEUwpLeGOkbsoPtwYYG7ssE', 'function': {'arguments': '{"query_str":"capital of France"}', 'name': 'wikipedia'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 100, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BWQQBNTkob2dB98x0XRs6shk1c2Gg', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-0876c5bd-4cf6-4c24-881e-b7a6c76749e7-0' tool_calls=[{'name': 'wikipedia', 'args': {'query_str': 'capital of France'}, 'id': 'call_qZEUwpLeGOkbsoPtwYYG7ssE', 'type': 'tool_call'}] usage_metadata={'input_tokens': 100, 'output_tokens': 17, 'total_tokens': 117, 'input_token_details': {'audio': 0, 'cache_re

In [11]:
from core.agents.tools.builtins.calculator import tool_function as calculator
from core.agents.tools.builtins.web_search import tool_function as web_search
llm_with_tools = llm.bind_tools([wikipedia, calculator, web_search])
sys_msg_tools = SystemMessage(
    content="You are a helpful research assistant that can use Wikipedia, a calculator, and web search."
)
human_msg = HumanMessage(
    content="What is the capital of France? What is 2 + 2? What is the weather like today in NYC?"
)
response = llm_with_tools.invoke([sys_msg_tools, human_msg])


In [14]:
from pprint import pprint
print(type(response))
pprint(response.tool_calls)

<class 'langchain_core.messages.ai.AIMessage'>
[{'args': {'query_str': 'capital of France'},
  'id': 'call_tkbgzuEyiuWAs9wPhei622u3',
  'name': 'wikipedia',
  'type': 'tool_call'},
 {'args': {'expression': '2 + 2'},
  'id': 'call_FTtuCCXh4GGiQ7oqn9BWJMK7',
  'name': 'calculator',
  'type': 'tool_call'},
 {'args': {'query': 'weather today in NYC', 'topic': 'news'},
  'id': 'call_iFcOr8ibYXjEN2DJUBAzDlbt',
  'name': 'web_search',
  'type': 'tool_call'}]
